In [1]:
# Import our dependencies
import os 
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from matplotlib import pyplot as plt
import plotly.express as px
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import tensorflow as tf
import math
from numpy import nan

In [2]:
#Import the dataset 
india_companies_df = pd.read_csv('Resources/related_databases/India-startup-funding-2021-full.csv')
# india_companies_df = companies_df.drop(['Portfolio Exits'],axis = 1)
india_companies_df.head(10)

,Month,Company/Brand,Founded,Headquarters,Sector,What it does,Founder/s,Investor/s,Amount ($),Stage
0,December,CollegeDekho,2015.0,Gurgaon,E-learning,"Collegedekho.com is Student’s Partner, Friend ...",Ruchir Arora,"Disrupt ADQ, QIC",35000000,Series B
1,December,BOX8,2012.0,Mumbai,Food & Beverages,India's Largest Desi Meals Brand,"Anshul Gupta, Amit Raj",Tiger Global,40000000,NaN
2,December,Simpl,2015.0,Bangalore,Consumer Services,Simpl empowers merchants to build trusted rela...,"Nitya, Chaitra Chidanand","Valar Ventures, IA Ventures",40000000,Series B
3,December,8i Ventures,2018.0,Mumbai,Venture Capital & Private Equity,8i is a Mumbai & Bangalore based early stage f...,Vikram Chachra,NaN,50000000,NaN
4,December,PayGlocal,2021.0,Bangalore,Financial Services,PayGlocal is a FinTech solving for global paym...,"Prachi Dharani, Rohit Sukhija, Yogesh Lokhande",Sequoia Capital India,4900000,Series A
5,December,Curefit,2016.0,Bangalore,"Health, Wellness & Fitness",Tata Digital-backed Curefit,"Mukesh Bansal, Ankit",Zomato,145000000,NaN
6,December,Pocket FM,2018.0,Bangalore,OTT,Pocket FM lets users discover and enjoy the gr...,"Rohan, Nishanth Srinivas, Prateek Dixit","Lightspeed, Times Group, Tanglin Venture Partners",22000000,Series B
7,December,CHARGE+ZONE,2018.0,Vadodara,Automotive,CHARGE+ZONE is a tech-driven EV Charging infra...,Kartikey Hariyani,Venture Catalysts,10000000,NaN
8,December,PlanetSpark,2017.0,Gurugram,Education Management,PlanetSpark is building the World Leader in Co...,"Maneesh Dhooper, Kunal Malik","Binny Bansal, Deep Kalra, Dr Ashish Gupta, Gok...",13000000,Series B
9,December,LenDenClub,2015.0,Mumbai,Financial Services,P2P lending platform delivering high returns o...,"Bhavin Patel, Dipesh Karki","Tuscan Ventures, Ohm Stock Brokers, Artha Vent...",10000000,Series A


In [3]:
dup = {}
drop = []
for x in range(0, len(india_companies_df['Company/Brand'])):
    if india_companies_df['Company/Brand'][x] not in dup.keys():
        dup[india_companies_df['Company/Brand'][x]] = [x]
    else:
        dup[india_companies_df['Company/Brand'][x]] += [x]
        drop.append(x)
len(dup)

1037

In [4]:
#dup

In [5]:
#need to have a counter of rows dropped to use iloc
rows_dropped = len(drop)#duplicates
#data = data.drop(labels=[1,15,20], axis=0)


In [6]:
india_df = india_companies_df.drop(labels=drop, axis=0)

In [7]:
def find_index(df, name): #all company names shud be unique
    y = 0
    for x in df['Company/Brand']:
        if x == name:
            return y
        y += 1

In [8]:
def group_stages(df):
    stage = {}
    #first_round = ["Series A","Series A+","Series A2"]
    y = 0
    for x in df['Stage']:
        if x not in stage.keys():
            stage[x] = [y]
        else:
            stage[x] += [y]
    
        y += 1
    return stage

In [9]:
def get_values(df):
    value = {}
    y = 0
    for x in india_df['Amount ($)']:
        if x not in value.keys():
            value[x] = [y]
        else:
            value[x] += [y]
    
        y += 1
    return value

In [10]:
#find the range of series b funding
#Clean the amount col firstttt
def funding(df,st, col):
    '''stage : type string
       col   : type string 
    '''
    mins = 0
    maxs = 0
    for y in stage[st]:
        if mins == 0 and maxs == 0:
            mins = int(df[col][y])
            maxs = int(df[col][y])
            
        elif int(df[col][y]) < mins :
            mins = int(df[col][y])
            
        elif maxs < int(df[col][y]):
            maxs = int(df[col][y])
            
        #print(india_companies_df[col][y]) # not required
    return (mins, maxs)

In [11]:
def change_stage_name(df, row,name):
    index = find_index(india_companies_df, df.iloc[row][1])
    india_df['Stage'][index] = name 

In [12]:
india_df

,Month,Company/Brand,Founded,Headquarters,Sector,What it does,Founder/s,Investor/s,Amount ($),Stage
0,December,CollegeDekho,2015.0,Gurgaon,E-learning,"Collegedekho.com is Student’s Partner, Friend ...",Ruchir Arora,"Disrupt ADQ, QIC",35000000,Series B
1,December,BOX8,2012.0,Mumbai,Food & Beverages,India's Largest Desi Meals Brand,"Anshul Gupta, Amit Raj",Tiger Global,40000000,NaN
2,December,Simpl,2015.0,Bangalore,Consumer Services,Simpl empowers merchants to build trusted rela...,"Nitya, Chaitra Chidanand","Valar Ventures, IA Ventures",40000000,Series B
3,December,8i Ventures,2018.0,Mumbai,Venture Capital & Private Equity,8i is a Mumbai & Bangalore based early stage f...,Vikram Chachra,NaN,50000000,NaN
4,December,PayGlocal,2021.0,Bangalore,Financial Services,PayGlocal is a FinTech solving for global paym...,"Prachi Dharani, Rohit Sukhija, Yogesh Lokhande",Sequoia Capital India,4900000,Series A
...,...,...,...,...,...,...,...,...,...,...
1203,January,BlackSoil,2016.0,Mumbai,FinTech,Blacksoil Advisory is an independent boutique ...,Ankur Bansal,Awign Enterprises,4000000,NaN
1204,January,HobSpace,2019.0,Mumbai,EdTech,HobSpace is a platform for extra-curricular ac...,"Priya Goel Sheth, Harsh Jain","Siddharth Bhaskar Shah, Upsparks",100000,Pre-series A1
1205,January,DcodeAI,2020.0,New Delhi,EdTech,EMPOWER LEARNERS TO BE FUTURE READY USING AI,"Kartik Sharma, Ashish Aggarwal",S Chand,500000,NaN
1206,January,Biocon Biologics,1978.0,Bangalore,BioTechnology,Biocon is a fully integrated pure play biosimi...,Kiran Mazumdar-Shaw,"Goldman Sachs, Tata Capital",70000000,NaN


In [13]:
len(india_companies_df) #indicates we have duplicates

1209

In [14]:
india_df = india_df.drop(columns = ['Founder/s','What it does'])

In [15]:
#for x in range india_companies_df['Amount ($)']:

In [16]:
india_df['Stage'].value_counts()

Seed             220
Pre-series A     128
Series A         111
Pre-seed          45
Series B          43
Series C          37
Series D          15
Series E          15
Debt              12
Pre-Series A       9
Pre-series B       8
Series F           4
Pre-series A1      4
Series G           2
Bridge             2
Pre-series         2
300000             2
Series D1          1
Seies A            1
1000000            1
6000000            1
Seed+              1
Early seed         1
Series F2          1
1200000            1
Series A2          1
Series F1          1
Name: Stage, dtype: int64

In [17]:
india_df['Amount ($)'].value_counts()

Undisclosed    116
1000000         74
2000000         57
3000000         43
5000000         36
              ... 
130000           1
4200000          1
2700000          1
550000           1
720000           1
Name: Amount ($), Length: 186, dtype: int64

In [18]:
value = get_values(india_df)

In [19]:
value.keys() # want to work through every non numeric value

dict_keys(['35000000', '40000000', '50000000', '4900000', '145000000', '22000000', '10000000', '13000000', '70000000', '100000000', '6700000', '30000000', '7000000', '5000000', '6600000', '32000000', '150000000', '24000000', '700000', 'Undisclosed', '200000', '1500000', '1300000', '1000000', '725000', '500000', '461000', 'Series C', 'Seed', '96000000', '4000000', '60000000', '20000000', '500000000', '266000000', '4500000', '1200000', '300000', '9500000', '325000000', '6500000', '1600000', '400000', '225000', '85000000', '235000', '260000', '2900000', '12000000', '53000000', '3500000', '3000000', '1100000', '150000', '86000000', '130000', '10000', '111000000', '37000000', '6000000', '265000', '76000000', '2000000', '3300000', '92000000', '600000', '100000', '17000000', '2500000', '135000000', '8000000', '15000000', '5500000', '155000', '2100000', '25000000', '21000000', '840000000', '248000000', '220000000', '4300000', '570000', '2200000', '4700000', '300000000', '52000000', '28000000',

In [20]:
value[nan]

[318, 355, 364, 714]

In [21]:
value['undisclosed']

[506, 507, 521, 524, 551]

In [22]:
value['Undisclosed']

[22,
 73,
 77,
 78,
 90,
 92,
 137,
 139,
 175,
 182,
 183,
 186,
 263,
 280,
 282,
 290,
 297,
 306,
 311,
 334,
 336,
 345,
 346,
 378,
 379,
 384,
 411,
 435,
 443,
 449,
 451,
 459,
 464,
 483,
 487,
 556,
 563,
 565,
 566,
 583,
 593,
 594,
 600,
 603,
 608,
 609,
 615,
 622,
 623,
 625,
 628,
 630,
 636,
 642,
 643,
 645,
 648,
 649,
 658,
 659,
 665,
 678,
 685,
 689,
 692,
 695,
 699,
 700,
 707,
 709,
 711,
 716,
 719,
 724,
 727,
 728,
 729,
 730,
 735,
 737,
 738,
 745,
 747,
 752,
 754,
 758,
 766,
 769,
 774,
 787,
 807,
 808,
 819,
 833,
 848,
 857,
 881,
 882,
 884,
 908,
 910,
 937,
 940,
 943,
 958,
 974,
 980,
 983,
 987,
 988,
 989,
 995,
 1016,
 1017,
 1019,
 1031]

In [23]:
#undisclosed,ah! Ventures,Pre-series A,'ITO Angel Network, LetsVenture','JITO Angel Network, LetsVenture',

In [24]:
value['JITO Angel Network, LetsVenture']

[717]

In [25]:
india_df.iloc[717]#df.iloc[0, df.columns.get_loc('COL_NAME')] = x

Month                                       June
Company/Brand                   Saarthi Pedagogy
Founded                                   2015.0
Headquarters                           Ahmadabad
Sector                                    EdTech
Investor/s                        Sushil Agarwal
Amount ($)       JITO Angel Network, LetsVenture
Stage                                    1000000
Name: 787, dtype: object

In [26]:
india_df['Investor/s'][787]

'Sushil Agarwal'

In [27]:
india_df['Investor/s'][787] = india_df['Investor/s'][787] + ', '+ india_df['Amount ($)'][787]

C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/4214881463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Investor/s'][787] = india_df['Investor/s'][787] + ', '+ india_df['Amount ($)'][787]


In [28]:
india_df['Investor/s'][787]

'Sushil Agarwal, JITO Angel Network, LetsVenture'

In [29]:
india_df.iloc[717]

Month                                                       June
Company/Brand                                   Saarthi Pedagogy
Founded                                                   2015.0
Headquarters                                           Ahmadabad
Sector                                                    EdTech
Investor/s       Sushil Agarwal, JITO Angel Network, LetsVenture
Amount ($)                       JITO Angel Network, LetsVenture
Stage                                                    1000000
Name: 787, dtype: object

In [30]:
india_df['Amount ($)'][787] = india_df['Stage'][787]

C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/3030007329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Amount ($)'][787] = india_df['Stage'][787]


In [31]:
india_df['Stage'][787] = math.nan

C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/3032095239.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Stage'][787] = math.nan


In [32]:
india_df.iloc[717]

Month                                                       June
Company/Brand                                   Saarthi Pedagogy
Founded                                                   2015.0
Headquarters                                           Ahmadabad
Sector                                                    EdTech
Investor/s       Sushil Agarwal, JITO Angel Network, LetsVenture
Amount ($)                                               1000000
Stage                                                        NaN
Name: 787, dtype: object

In [33]:
value['Series C']

[30]

In [34]:
india_df.iloc[30]

Month                                                     December
Company/Brand                                   Fullife Healthcare
Founded                                                     2009.0
Headquarters                                 Pharmaceuticals #REF!
Sector           Primary Business is Development and Manufactur...
Investor/s                                                22000000
Amount ($)                                                Series C
Stage                                                          NaN
Name: 30, dtype: object

In [35]:
india_df['Stage'][30] = india_df['Amount ($)'][30]
india_df['Amount ($)'][30] = india_df['Investor/s'][30]
india_df['Investor/s'][30] = math.nan

C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/321561579.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Stage'][30] = india_df['Amount ($)'][30]
C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/321561579.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Amount ($)'][30] = india_df['Investor/s'][30]
C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/321561579.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [36]:
india_df.iloc[30]

Month                                                     December
Company/Brand                                   Fullife Healthcare
Founded                                                     2009.0
Headquarters                                 Pharmaceuticals #REF!
Sector           Primary Business is Development and Manufactur...
Investor/s                                                     NaN
Amount ($)                                                22000000
Stage                                                     Series C
Name: 30, dtype: object

In [37]:
value['Seed']

[31, 366]

In [38]:
india_df.iloc[31]

Month                                                     December
Company/Brand                                              MoEVing
Founded                                                     2021.0
Headquarters                                        Gurugram #REF!
Sector           MoEVing is India's only Electric Mobility focu...
Investor/s                                                 5000000
Amount ($)                                                    Seed
Stage                                                          NaN
Name: 45, dtype: object

In [39]:
india_df['Company/Brand'][45]

'MoEVing'

In [40]:
india_df['Stage'][45] = india_df['Amount ($)'][45]
india_df['Amount ($)'][45] = india_df['Investor/s'][45]
india_df['Investor/s'][45] = math.nan

C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/3682406484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Stage'][45] = india_df['Amount ($)'][45]
C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/3682406484.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Amount ($)'][45] = india_df['Investor/s'][45]
C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/3682406484.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [41]:
india_df.iloc[31]

Month                                                     December
Company/Brand                                              MoEVing
Founded                                                     2021.0
Headquarters                                        Gurugram #REF!
Sector           MoEVing is India's only Electric Mobility focu...
Investor/s                                                     NaN
Amount ($)                                                 5000000
Stage                                                         Seed
Name: 45, dtype: object

In [42]:
india_df.iloc[366]

Month                           September
Company/Brand                   Godamwale
Founded                            2016.0
Headquarters                       Mumbai
Sector           Logistics & Supply Chain
Investor/s                  1000000 #REF!
Amount ($)                           Seed
Stage                                 NaN
Name: 383, dtype: object

In [43]:
india_df['Company/Brand'][383]

'Godamwale'

In [44]:
india_df['Stage'][383] = india_df['Amount ($)'][383]
india_df['Amount ($)'][383] = india_df['Investor/s'][383][:7]
india_df['Investor/s'][383] = math.nan

C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/3552351949.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Stage'][383] = india_df['Amount ($)'][383]
C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/3552351949.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Amount ($)'][383] = india_df['Investor/s'][383][:7]
C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/3552351949.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [45]:
india_df.iloc[366]

Month                           September
Company/Brand                   Godamwale
Founded                            2016.0
Headquarters                       Mumbai
Sector           Logistics & Supply Chain
Investor/s                            NaN
Amount ($)                        1000000
Stage                                Seed
Name: 383, dtype: object

In [46]:
value['Upsparks']

[434]

In [47]:
india_df.iloc[434]#455

Month                                 August
Company/Brand                        FanPlay
Founded                               2020.0
Headquarters                  Computer Games
Sector                        Computer Games
Investor/s       Pritesh Kumar, Bharat Gupta
Amount ($)                          Upsparks
Stage                                1200000
Name: 455, dtype: object

In [48]:
india_df['Investor/s'][455]

'Pritesh Kumar, Bharat Gupta'

In [49]:
india_df['Investor/s'][455] = india_df['Investor/s'][455] + " , " + india_df['Amount ($)'][455]
india_df['Amount ($)'][455] = india_df['Stage'][455]
india_df['Stage'][455] = math.nan

C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/3242314008.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Investor/s'][455] = india_df['Investor/s'][455] + " , " + india_df['Amount ($)'][455]
C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/3242314008.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Amount ($)'][455] = india_df['Stage'][455]
C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/3242314008.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [50]:
india_df.iloc[434]#455

Month                                            August
Company/Brand                                   FanPlay
Founded                                          2020.0
Headquarters                             Computer Games
Sector                                   Computer Games
Investor/s       Pritesh Kumar, Bharat Gupta , Upsparks
Amount ($)                                      1200000
Stage                                               NaN
Name: 455, dtype: object

In [51]:
value['undisclosed']

[506, 507, 521, 524, 551]

In [52]:
india_df.iloc[506]

Month                             August
Company/Brand                   AyuRythm
Founded                           2017.0
Headquarters                   Bangalore
Sector                        Healthcare
Investor/s       SucSEED Indovation Fund
Amount ($)                   undisclosed
Stage                               Seed
Name: 549, dtype: object

In [53]:
index = find_index(india_companies_df, india_df.iloc[506][1])
index

549

In [54]:
india_df['Company/Brand'][549]

'AyuRythm'

In [55]:
for y in value['undisclosed']:
    print(india_df.iloc[y])
    print("@")

Month                             August
Company/Brand                   AyuRythm
Founded                           2017.0
Headquarters                   Bangalore
Sector                        Healthcare
Investor/s       SucSEED Indovation Fund
Amount ($)                   undisclosed
Stage                               Seed
Name: 549, dtype: object
@
Month                                                       August
Company/Brand                                               Enguru
Founded                                                     2014.0
Headquarters                                             Bangalore
Sector                                                      EdTech
Investor/s       Potencia Ventures, Bisk Ventures, LetsVenture,...
Amount ($)                                             undisclosed
Stage                                                 Pre-series A
Name: 550, dtype: object
@
Month                                                       August
Company/Brand  

In [56]:
for y in value['ah! Ventures']:
    print(india_df.iloc[y])
    print("@")

Month                    July
Company/Brand     Little Leap
Founded                2020.0
Headquarters        New Delhi
Sector                 EdTech
Investor/s       Vishal Gupta
Amount ($)       ah! Ventures
Stage                  300000
Name: 648, dtype: object
@


In [57]:
value['ah! Ventures']

[597]

In [58]:
index  = find_index(india_companies_df,india_df.iloc[597][1] )

In [59]:
index

648

In [60]:
india_df['Investor/s'][index] = india_df['Investor/s'][index] + " , " + india_df['Amount ($)'][index]
india_df['Amount ($)'][index] = india_df['Stage'][index]
india_df['Stage'][index] = math.nan

C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/2983144170.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Investor/s'][index] = india_df['Investor/s'][index] + " , " + india_df['Amount ($)'][index]
C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/2983144170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Amount ($)'][index] = india_df['Stage'][index]
C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/2983144170.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

In [61]:
india_df.iloc[597]

Month                                   July
Company/Brand                    Little Leap
Founded                               2020.0
Headquarters                       New Delhi
Sector                                EdTech
Investor/s       Vishal Gupta , ah! Ventures
Amount ($)                            300000
Stage                                    NaN
Name: 648, dtype: object

In [62]:
value['Pre-series A']

[602]

In [63]:
india_df.iloc[602]

Month                    July
Company/Brand       AdmitKard
Founded                2016.0
Headquarters            Noida
Sector                 EdTech
Investor/s            1000000
Amount ($)       Pre-series A
Stage                     NaN
Name: 655, dtype: object

In [64]:
index  = find_index(india_companies_df,india_df.iloc[602][1])

In [65]:
india_df['Stage'][index] = india_df['Amount ($)'][index]
india_df['Amount ($)'][index] = india_df['Investor/s'][index]
india_df['Investor/s'][index] = math.nan

C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/103159855.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Stage'][index] = india_df['Amount ($)'][index]
C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/103159855.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Amount ($)'][index] = india_df['Investor/s'][index]
C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/103159855.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [66]:
india_df.iloc[602]

Month                    July
Company/Brand       AdmitKard
Founded                2016.0
Headquarters            Noida
Sector                 EdTech
Investor/s                NaN
Amount ($)            1000000
Stage            Pre-series A
Name: 655, dtype: object

In [67]:
value['ITO Angel Network, LetsVenture']

[607]

In [68]:
india_df.iloc[607]

Month                                       July
Company/Brand                              BHyve
Founded                                   2020.0
Headquarters                              Mumbai
Sector                           Human Resources
Investor/s       Omkar Pandharkame, Ketaki Ogale
Amount ($)        ITO Angel Network, LetsVenture
Stage                                     300000
Name: 661, dtype: object

In [69]:
index = find_index(india_companies_df, india_df.iloc[607][1])

In [70]:
india_df['Investor/s'][index] = india_df['Investor/s'][index] + " , " + india_df['Amount ($)'][index]
india_df['Amount ($)'][index] = india_df['Stage'][index]
india_df['Stage'][index] = math.nan

C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/2983144170.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Investor/s'][index] = india_df['Investor/s'][index] + " , " + india_df['Amount ($)'][index]
C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/2983144170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Amount ($)'][index] = india_df['Stage'][index]
C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/2983144170.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

In [71]:
india_df.iloc[607]

Month                                                         July
Company/Brand                                                BHyve
Founded                                                     2020.0
Headquarters                                                Mumbai
Sector                                             Human Resources
Investor/s       Omkar Pandharkame, Ketaki Ogale , ITO Angel Ne...
Amount ($)                                                  300000
Stage                                                          NaN
Name: 661, dtype: object

In [72]:
#dont adjusting these values
value = get_values(india_df)

In [73]:
value.keys() #we will come back for nan, undisclosed and Undiclosed

dict_keys(['35000000', '40000000', '50000000', '4900000', '145000000', '22000000', '10000000', '13000000', '70000000', '100000000', '6700000', '30000000', '7000000', '5000000', '6600000', '32000000', '150000000', '24000000', '700000', 'Undisclosed', '200000', '1500000', '1300000', '1000000', '725000', '500000', '461000', '96000000', '4000000', '60000000', '20000000', '500000000', '266000000', '4500000', '1200000', '300000', '9500000', '325000000', '6500000', '1600000', '400000', '225000', '85000000', '235000', '260000', '2900000', '12000000', '53000000', '3500000', '3000000', '1100000', '150000', '86000000', '130000', '10000', '111000000', '37000000', '6000000', '265000', '76000000', '2000000', '3300000', '92000000', '600000', '100000', '17000000', '2500000', '135000000', '8000000', '15000000', '5500000', '155000', '2100000', '25000000', '21000000', '840000000', '248000000', '220000000', '4300000', '570000', '2200000', '4700000', '300000000', '52000000', '28000000', '260000000', '14000

In [74]:
#find indices of nan and Undisclosed and try to estimate value based on Stage
#clean stage col

In [75]:
stage = group_stages(india_df)

In [76]:
len(stage.keys())

26

In [77]:
#len(stage.keys()) = 26
stage.keys()

dict_keys(['Series B', nan, 'Series A', 'Series D', 'Pre-series A', 'Seed', 'Series C', 'Series E', 'Series G', 'Pre-seed', 'Pre-series B', 'Series F', 'Pre-series', 'Debt', 'Pre-Series A', 'Series A2', 'Bridge', 'Pre-series A1', nan, 'Seed+', 'Series F2', 'Early seed', 'Series D1', '6000000', 'Seies A', 'Series F1'])

In [78]:
len(stage[nan])

364

In [79]:
india_df.iloc[402]

Month                    September
Company/Brand             Vidyakul
Founded                     2017.0
Headquarters              Gurugram
Sector                      EdTech
Investor/s       We Founder Circle
Amount ($)                  500000
Stage                       Bridge
Name: 419, dtype: object

In [80]:
print("Pre-series A" == "Pre-Series A")

False


In [81]:
a_list = [385]
a_list.extend(stage['Pre-Series A'])
a_list.extend(stage['Pre-series A1'])
a_list.extend(stage['Seies A'])

In [82]:
a_list

[385, 167, 181, 191, 193, 194, 197, 210, 447, 467, 409, 612, 657, 1033, 743]

In [83]:
stage['Pre-Series A']

[167, 181, 191, 193, 194, 197, 210, 447, 467]

In [84]:
stage['Pre-series A1']

[409, 612, 657, 1033]

In [85]:
for x in a_list:
    change_stage_name(india_df, x, 'Pre-series A')

C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/3857762600.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Stage'][index] = name


In [86]:
india_df.iloc[153]
index = find_index(india_companies_df,india_df.iloc[153][1] )

In [87]:
#change_stage_b(df,row)
change_stage_name(india_df, 153, 'Pre-series B')

C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/3857762600.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Stage'][index] = name


In [88]:
stage['Pre-series']#both accounted for

[153, 385]

In [89]:
stage['6000000']

[714]

In [90]:
india_df.iloc[714]

Month                              June
Company/Brand              MYRE Capital
Founded                          2020.0
Headquarters                     Mumbai
Sector           Commercial Real Estate
Investor/s                  Aryaman Vir
Amount ($)                          NaN
Stage                           6000000
Name: 784, dtype: object

In [91]:
index = find_index(india_companies_df, india_df.iloc[714][1])
index

784

In [92]:
india_df['Amount ($)'][index] = india_df['Stage'][index]
india_df['Stage'][index] = math.nan

C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/834814723.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Amount ($)'][index] = india_df['Stage'][index]
C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/834814723.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Stage'][index] = math.nan


In [93]:
india_df.iloc[714]

Month                              June
Company/Brand              MYRE Capital
Founded                          2020.0
Headquarters                     Mumbai
Sector           Commercial Real Estate
Investor/s                  Aryaman Vir
Amount ($)                      6000000
Stage                               NaN
Name: 784, dtype: object

In [94]:
stage = group_stages(india_df)
stage['Debt']

[161, 329, 800, 851, 861, 877, 956, 966, 976, 979, 1006, 1013]

In [95]:
stage['Debt'].remove(800)

In [96]:
b_group = stage['Series B']
b_group.extend(stage['Debt'])
for x in b_group:
    change_stage_name(india_df, x, 'Series B')

C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/3857762600.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Stage'][index] = name


In [97]:
a_group = stage['Series A']
a_group.extend(stage['Series A2'])
for x in a_group:
    change_stage_name(india_df, x, 'Series A')

C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/3857762600.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Stage'][index] = name


In [98]:
c_group = stage['Series E']
c_group.extend(stage['Series G'])
c_group.extend(stage['Series F'])
c_group.extend(stage['Series F2'])
c_group.extend(stage['Series D1'])
c_group.extend(stage['Series F1'])
c_group.extend(stage['Series D'])
c_group.append(800)

for x in c_group:
    change_stage_name(india_df, x, 'Series C')

C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/3857762600.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Stage'][index] = name


In [99]:
pre_seed_group = stage['Pre-seed']
pre_seed_group.extend(stage['Seed+'])
pre_seed_group.extend(stage['Early seed'])
pre_seed_group.extend(stage['Bridge'])
for x in pre_seed_group:
    change_stage_name(india_df, x, 'Pre-seed')

C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/3857762600.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Stage'][index] = name


In [100]:
stage = group_stages(india_df)

In [101]:
stage.keys()

dict_keys(['Series B', nan, 'Series A', 'Series C', 'Pre-series A', 'Seed', 'Pre-seed', 'Pre-series B', nan])

In [102]:
other = [nan, 'Undisclosed', 'undisclosed']
drop_row = []
isnan = []
for x in stage[nan]:
    val = india_df.iloc[x][6]
    if val in other:
        drop_row.append(x)
    #if math.isnan(india_df.iloc[x][6]):
        #isnan.append(x)
    #elif india_df.iloc[x][6]
    print(india_df.iloc[x][6])
#drop_row #DROP ROWS that have NEITHER a stage nor amount of funding NAN FOR BOTH

40000000
50000000
145000000
10000000
6700000
200000
500000
5000000
20000000
500000000
1200000
300000
200000
325000000
1600000
400000
5000000
100000000
10000000
200000
3000000
Undisclosed
150000
Undisclosed
Undisclosed
130000
37000000
265000
1000000
76000000
Undisclosed
Undisclosed
2000000
150000
92000000
10000000
400000
2000000
150000
4500000
17000000
3000000
2000000
2500000
20000000
15000000
5000000
Undisclosed
Undisclosed
5500000
155000
30000000
400000
40000000
840000000
10000000
12000000
4300000
4700000
300000000
1300000
4000000
Undisclosed
Undisclosed
Undisclosed
2000000
19000000
200000000
7500000
600000000
400000
24000000
2300000
4000000
60000000
2200000
100000
1000000
50000000
500000
400000
100000000
1100000
10000000
500000
10000000
Undisclosed
8000000
5000000
10000000
70000000
Undisclosed
570000000
Undisclosed
7000000
150000
2700000
400000
125000000
10000000
Undisclosed
30000000
50000000
nan
1200000
200000
25000000
Undisclosed
700000
30000000
Undisclosed
200000
1000000
4000000
n

In [103]:
#data = data.drop(labels=[1,15,20], axis=0)
drop_row.remove(311)
drop_row.remove(645)
india_df = india_df.drop(labels=drop_row, axis = 0)

In [104]:
india_df

,Month,Company/Brand,Founded,Headquarters,Sector,Investor/s,Amount ($),Stage
0,December,CollegeDekho,2015.0,Gurgaon,E-learning,"Disrupt ADQ, QIC",35000000,Series B
1,December,BOX8,2012.0,Mumbai,Food & Beverages,Tiger Global,40000000,NaN
2,December,Simpl,2015.0,Bangalore,Consumer Services,"Valar Ventures, IA Ventures",40000000,Series B
3,December,8i Ventures,2018.0,Mumbai,Venture Capital & Private Equity,NaN,50000000,NaN
4,December,PayGlocal,2021.0,Bangalore,Financial Services,Sequoia Capital India,4900000,Series A
...,...,...,...,...,...,...,...,...
1203,January,BlackSoil,2016.0,Mumbai,FinTech,Awign Enterprises,4000000,NaN
1204,January,HobSpace,2019.0,Mumbai,EdTech,"Siddharth Bhaskar Shah, Upsparks",100000,Pre-series A
1205,January,DcodeAI,2020.0,New Delhi,EdTech,S Chand,500000,NaN
1206,January,Biocon Biologics,1978.0,Bangalore,BioTechnology,"Goldman Sachs, Tata Capital",70000000,NaN


In [105]:
india_df.dropna(subset=['Amount ($)','Stage'],how='all')

,Month,Company/Brand,Founded,Headquarters,Sector,Investor/s,Amount ($),Stage
0,December,CollegeDekho,2015.0,Gurgaon,E-learning,"Disrupt ADQ, QIC",35000000,Series B
1,December,BOX8,2012.0,Mumbai,Food & Beverages,Tiger Global,40000000,NaN
2,December,Simpl,2015.0,Bangalore,Consumer Services,"Valar Ventures, IA Ventures",40000000,Series B
3,December,8i Ventures,2018.0,Mumbai,Venture Capital & Private Equity,NaN,50000000,NaN
4,December,PayGlocal,2021.0,Bangalore,Financial Services,Sequoia Capital India,4900000,Series A
...,...,...,...,...,...,...,...,...
1203,January,BlackSoil,2016.0,Mumbai,FinTech,Awign Enterprises,4000000,NaN
1204,January,HobSpace,2019.0,Mumbai,EdTech,"Siddharth Bhaskar Shah, Upsparks",100000,Pre-series A
1205,January,DcodeAI,2020.0,New Delhi,EdTech,S Chand,500000,NaN
1206,January,Biocon Biologics,1978.0,Bangalore,BioTechnology,"Goldman Sachs, Tata Capital",70000000,NaN


In [106]:
stage = group_stages(india_df)
#stage['Debt']

In [107]:
def stage_avg(name):
    amount = len(stage[name])
    tot = 0
    unknown = [nan, 'Undisclosed', 'undisclosed']
    for x in stage[name]:
        if india_df.iloc[x][6] not in unknown:
            
            tot += int(india_df.iloc[x][6])
        
    return int(tot/amount)

In [108]:
#temporarily remove nan, undisclosed and Undiclosed to get avgs
Final_stages = ['Series B' 'Series A', 'Series C', 'Pre-series A', 'Seed', 'Pre-seed', 'Pre-series B', 'Debt', 'Bridge']
unknown = [nan, 'Undisclosed', 'undisclosed']
stage_copy = stage.copy()
drop_rows = []
for x in stage_copy.keys():
    for y in stage_copy[x]:
        amount = india_df.iloc[y][6]
        curr_stage = india_df.iloc[y][7]
        if amount in unknown and curr_stage in unknown:
            india_df.iloc[y][6] = math.nan
            india_df.iloc[y][7] = math.nan
            drop_rows.append(y)

            #print(curr_stage)
#drop_rows

C:\Users\Acer\anaconda3\lib\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [109]:
#drop rows where both amount and stage is nan
india_df.drop(india_df.index[drop_rows], inplace=True)

In [110]:
stage = group_stages(india_df)
#stage['Debt']

In [111]:
india_df.iloc[762]

Month                              April
Company/Brand                   Safexpay
Founded                           2017.0
Headquarters                       Thane
Sector                           FinTech
Investor/s       T Choithrams BVI, Ardor
Amount ($)                       6000000
Stage                                NaN
Name: 960, dtype: object

In [112]:
stage_copy = stage.copy()

In [113]:
stage_copy.keys()

dict_keys(['Series B', nan, 'Series A', 'Series C', 'Pre-series A', 'Seed', 'Pre-seed', 'Pre-series B', nan])

In [114]:
stage = group_stages(india_df)

In [115]:
stage.keys()

dict_keys(['Series B', nan, 'Series A', 'Series C', 'Pre-series A', 'Seed', 'Pre-seed', 'Pre-series B', nan])

In [116]:
india_df

,Month,Company/Brand,Founded,Headquarters,Sector,Investor/s,Amount ($),Stage
0,December,CollegeDekho,2015.0,Gurgaon,E-learning,"Disrupt ADQ, QIC",35000000,Series B
1,December,BOX8,2012.0,Mumbai,Food & Beverages,Tiger Global,40000000,NaN
2,December,Simpl,2015.0,Bangalore,Consumer Services,"Valar Ventures, IA Ventures",40000000,Series B
3,December,8i Ventures,2018.0,Mumbai,Venture Capital & Private Equity,NaN,50000000,NaN
4,December,PayGlocal,2021.0,Bangalore,Financial Services,Sequoia Capital India,4900000,Series A
...,...,...,...,...,...,...,...,...
1203,January,BlackSoil,2016.0,Mumbai,FinTech,Awign Enterprises,4000000,NaN
1204,January,HobSpace,2019.0,Mumbai,EdTech,"Siddharth Bhaskar Shah, Upsparks",100000,Pre-series A
1205,January,DcodeAI,2020.0,New Delhi,EdTech,S Chand,500000,NaN
1206,January,Biocon Biologics,1978.0,Bangalore,BioTechnology,"Goldman Sachs, Tata Capital",70000000,NaN


In [117]:
stage.keys()

dict_keys(['Series B', nan, 'Series A', 'Series C', 'Pre-series A', 'Seed', 'Pre-seed', 'Pre-series B', nan])

In [122]:
def find_closest(num):
    closest = math.nan
    diff = 2**31 - 1
    stages = ['Series B', 'Series A', 'Series C', 'Pre-series A', 'Seed', 'Pre-seed', 'Pre-series B']
    list_of_avgs = {'Series B':[], 'Series A':[], 'Series C':[], 'Pre-series A':[], 'Seed':[], 'Pre-seed':[], 'Pre-series B':[]}
    for x in stages:
        list_of_avgs[x] = stage_avg(x)
        if abs(num - list_of_avgs[x]) < diff:
            diff = abs(num - list_of_avgs[x])
            closest = x
    return x
        #print(stage_avg(x))

In [120]:
list_of_avgs

{'Series B': 2978696078,
 'Series A': 11556250,
 'Series C': 91862500,
 'Pre-series A': 1754669,
 'Seed': 2357929,
 'Pre-seed': 634767,
 'Pre-series B': 7944444}

In [124]:
for x in stage[nan]:
    val = india_df.iloc[x][6]
    stage_curr = find_closest(int(val))
    change_stage_name(india_df, x, stage_curr )

C:\Users\Acer\AppData\Local\Temp/ipykernel_18984/3857762600.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['Stage'][index] = name


In [125]:
india_df

,Month,Company/Brand,Founded,Headquarters,Sector,Investor/s,Amount ($),Stage
0,December,CollegeDekho,2015.0,Gurgaon,E-learning,"Disrupt ADQ, QIC",35000000,Series B
1,December,BOX8,2012.0,Mumbai,Food & Beverages,Tiger Global,40000000,Pre-series B
2,December,Simpl,2015.0,Bangalore,Consumer Services,"Valar Ventures, IA Ventures",40000000,Series B
3,December,8i Ventures,2018.0,Mumbai,Venture Capital & Private Equity,NaN,50000000,Pre-series B
4,December,PayGlocal,2021.0,Bangalore,Financial Services,Sequoia Capital India,4900000,Series A
...,...,...,...,...,...,...,...,...
1203,January,BlackSoil,2016.0,Mumbai,FinTech,Awign Enterprises,4000000,Pre-series B
1204,January,HobSpace,2019.0,Mumbai,EdTech,"Siddharth Bhaskar Shah, Upsparks",100000,Pre-series A
1205,January,DcodeAI,2020.0,New Delhi,EdTech,S Chand,500000,Pre-series B
1206,January,Biocon Biologics,1978.0,Bangalore,BioTechnology,"Goldman Sachs, Tata Capital",70000000,Pre-series B


In [ ]:
india_df.to_csv('')

In [ ]:
#FINAL TABLE #df.to_csv('file_name.csv')

In [ ]:
#change_stage_name(india_df, x, 'Pre-seed')

In [ ]:
india_companies_df['Stage'].value_counts()

In [ ]:
#data = data.drop(labels=[1,15,20], axis=0)
#india_df = india_df.drop(labels=[44], axis = 0)


In [ ]:
len(india_companies_df)
#data = data.drop(labels=[1,15,20], axis=0)

In [ ]:
len(india_companies_df)

In [ ]:
india_companies_df['Sector'].value_counts()

In [ ]:
sector = {}
for x in range(0, len(india_companies_df['Sector'])):
    if india_companies_df['Sector'][x] not in sector:
        sector[x] = india_companies_df['Sector'][x]
sector
len(sector)

In [ ]:
sector_types = []
for x in india_companies_df['Sector']:
    if x not in sector_types:
        sector_types.append(x)
sector_types
len(sector_types)

In [ ]:
india_companies_df['Sector'][2]

In [ ]:
for x in india_companies_df['Sector']:
    print(x)

In [ ]:
india_companies_df['Month'].value_counts()

In [ ]:
india_companies_df = india_companies_df[india_companies_df['Founded']>= 2000]
india_companies_df = india_companies_df[india_companies_df['Amount ($)']!= 0]

india_companies_df['Amount ($)'].value_counts()

In [ ]:
for column in india_companies_df.columns:
    print (f"Column {column} has {india_companies_df[column].isnull().sum()} null values")

In [ ]:
#Replace NaN's

india_companies_df["Stage"] = india_companies_df["Stage"].replace(np.nan,"Other")
india_companies_df["Headquarters"] = india_companies_df["Headquarters"].replace(np.nan,"Other")
india_companies_df["Investor/s"] = india_companies_df["Investor/s"].replace(np.nan,"Other")
# india_companies_df["Founder/s"] = india_companies_df["Founder/s"].replace(np.nan,"Other")
india_companies_df = india_companies_df.drop(columns = ['Founder/s','What it does'])

# #Verify binning was successful 
# india_companies_df["Stage"].value_counts()

In [ ]:
india_companies_df.set_index('Company/Brand',inplace = True)
india_companies_df.head(10)


In [ ]:
india_encoded = pd.get_dummies(india_companies_df,columns = ['Month','Headquarters','Sector','Investor/s','Stage'])
india_encoded.head()